In [78]:
import glob
import os
from PIL import Image
import sqlite3
import pandas as pd
import shutil

In [88]:
FRAME_ID = 27937
TILES_DIR = '/Users/darylwilding-mcbride/Downloads/tiles-inf-method/all'
ANIMATION_FRAMES_DIR = '{}/frames'.format(TILES_DIR)
VIDEO_DIR = '{}/video'.format(TILES_DIR)
TILE_START = 20
TILE_END = 60
CONVERTED_DATABASE = '/Users/darylwilding-mcbride/Downloads/HeLa_20KInt-rt-3000-3600-denoised/HeLa_20KInt.sqlite'
RT_LOWER = 3000
RT_UPPER = 3060
MS1_CE = 10

In [89]:
# initialise the directories required
if os.path.exists(ANIMATION_FRAMES_DIR):
    shutil.rmtree(ANIMATION_FRAMES_DIR)
os.makedirs(ANIMATION_FRAMES_DIR)

# if os.path.exists(VIDEO_DIR):
#     shutil.rmtree(VIDEO_DIR)
# os.makedirs(VIDEO_DIR)

In [90]:
db_conn = sqlite3.connect(CONVERTED_DATABASE)
ms1_frame_properties_df = pd.read_sql_query("select frame_id,retention_time_secs from frame_properties where retention_time_secs >= {} and retention_time_secs <= {} and collision_energy == {}".format(RT_LOWER, RT_UPPER, MS1_CE), db_conn)
db_conn.close()

In [91]:
for idx in range(len(ms1_frame_properties_df)):
    frame_id = int(ms1_frame_properties_df.iloc[idx].frame_id)
    file_list = []
    for tile_id in range(TILE_START, TILE_END+1):
        file_list.append(glob.glob("{}/frame-{}-tile-{}-mz-*.png".format(TILES_DIR, frame_id, tile_id))[0])
    images = list(map(Image.open, file_list))
    
    widths, heights = zip(*(i.size for i in images))
    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for im in images:
      new_im.paste(im, (x_offset,0))
      x_offset += im.size[0]

    new_im.save('{}/frame-{:04d}.png'.format(ANIMATION_FRAMES_DIR, idx))